%# Advent of Code 2025

## Day 1: Secret Entrance
### Part 1

In [2]:
import java.io.File

fun dayOnePartOne(fileName: String): String {
    var current = 50
    var zeroCount = 0
    File(fileName).useLines { lines ->
        lines.forEach { line ->
            val dir = line.first()
            val count = line.drop(1).toInt()

            if (dir == 'R') {
                current += count
            } else {
                current -= count
            }

            if (current % 100 == 0) {
                zeroCount++
            }
        }
    }
    return zeroCount.toString()
}

println("The test data result should be 3. Actual value: ${dayOnePartOne("testData/day1.txt")}")
println("The result using the actual value is: ${dayOnePartOne("data/day1.txt")}")

The test data result should be 3. Actual value: 3
The result using the actual value is: 1139


### Part 2

In [7]:
fun dayOnePartTwo(fileName: String): String {
    var current = 50
    var zeroCount = 0
    File(fileName).useLines { lines ->
        lines.forEach { line ->
            val direction = line.first()
            val count = line.drop(1).toInt()

            if (direction == 'R') {
                current += count % 100
                if (current > 100) {
                    zeroCount++
                    current = current - 100
                }
                zeroCount += count / 100
            } else {
                val prevWasZero = current == 0
                current -= count % 100
                if (current < 0) {
                    if (!prevWasZero) {
                        zeroCount++
                    }
                    current = 100 + current
                }
                zeroCount += count / 100
            }

            if (current % 100 == 0) {
                zeroCount++
                current = 0
            }
        }
    }
    return zeroCount.toString()
}

println("The test data result should be 6. Actual value: ${dayOnePartTwo("testData/day1.txt")}")
println("The result using the actual value is: ${dayOnePartTwo("data/day1.txt")}")

The test data result should be 6. Actual value: 6
The result using the actual value is: 6684
